# Evolution of the gas metal abundance

In [5]:
import numpy as np
from scipy.stats import binned_statistic
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [1]:
from auriga.snapshot import Snapshot
from auriga.settings import Settings
from auriga.paths import Paths
from auriga.parser import parse
from auriga.support import make_snapshot_number
from auriga.images import figure_setup

/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/setuptools/_importlib.py:23: UserWarning: `importlib-metadata` version is incompatible with `setuptools`.
This problem is likely to be solved by installing an updated version of `importlib-metadata`.
  warnings.warn(msg)  # Ensure a descriptive message is shown.


In [2]:
N_BINS_ABUNDANCE: int = 50
N_BINS_TIME: int = 28

In [7]:
def get_gas_abundances(simulation: str, of: str, to: str,
                       max_radius: float) -> list:
    s = Snapshot(simulation=simulation, loadonlytype=[0])
    s.add_metal_abundance(of=of, to=to)
    s.add_extra_coordinates()
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)
    is_inner = s.rho <= max_radius
    abundance = s.metal_abundance[f"{of}/{to}"][is_main_obj & is_inner]
    abundance = abundance[np.isfinite(abundance)]
    return list(abundance)

In [4]:
def calculate_gas_abundance_evolution(
        simulation: str, of: str, to: str,
        abundance_min: float, abundance_max: float,
        max_radius: float) -> None:
    settings = Settings()
    galaxy, rerun, resolution = parse(simulation)
    paths = Paths(galaxy, rerun, resolution)
    n_snapshots = make_snapshot_number(rerun, resolution)
    snapshot_times = pd.read_csv(
        f"../results/{simulation}/temporal_data.csv")["Time_Gyr"]

    accumulated_abundances = []
    data = np.nan * np.ones((N_BINS_ABUNDANCE, N_BINS_TIME))
    time_ref_idx = 0
    for i in range(settings.first_snap, n_snapshots):
        if snapshot_times[i] > (time_ref_idx + 1) * 0.5:
            # Create histogram
            binned_abundance = np.histogram(
                accumulated_abundances, density=True,
                range=(abundance_min, abundance_max),
                bins=N_BINS_ABUNDANCE)[0]
            data[:, time_ref_idx] = binned_abundance

            # Reset loop variables
            accumulated_abundances = []
            time_ref_idx += 1
        accumulated_abundances += get_gas_abundances(
            simulation + f"_s{i}", of, to, max_radius)

    np.save(f"../results/{simulation}/gas_{of}{to}_abundance_evol.npy", data)

In [34]:
def make_plot(data: np.ndarray, xrange: tuple, yrange: tuple,
              ylabel: str, label: str, filename: str):
    fig = plt.figure(figsize=(3.0, 2.5))
    gs = fig.add_gridspec(nrows=1, ncols=1, hspace=0.0, wspace=0.0)
    ax = gs.subplots(sharex=True, sharey=True)

    ax.set_xlim(xrange)
    ax.set_xlabel("Time [Gyr]")

    ax.set_ylim(yrange)
    ax.set_ylabel(ylabel)

    ax.tick_params(which='both', direction="in")
    ax.label_outer()

    im = ax.imshow(data, cmap="viridis", aspect="auto", origin="lower",
                   extent=(xrange[0], xrange[1], yrange[0], yrange[1]),
                   norm=LogNorm(0.001, vmax=5.0), rasterized=True,
                   interpolation="none")

    cbar = fig.colorbar(im, ax=ax, label="PDF", ticks=[0.001, 0.01, 0.1, 1.0])
    cbar.ax.set_yticklabels([0.001, 0.01, 0.1, 1.0])

    ax.text(x=0.05, y=0.95, size=8.0, s=r"$\texttt{" + label + r"}$",
            ha='left', va='top', transform=ax.transAxes)

    fig.savefig(f"../images/gas_abundance/{filename}.pdf")
    plt.close(fig)

In [35]:
figure_setup()

### Calculate Data

In [ ]:
calculate_gas_abundance_evolution(
    simulation="au6_or_l4", of="Fe", to='H',
    abundance_min=-2.5, abundance_max=1.5, max_radius=30.0)

In [39]:
calculate_gas_abundance_evolution(
    simulation="au6_or_l4", of='O', to="Fe",
    abundance_min=0.0, abundance_max=1.0, max_radius=30.0)

### Figures

In [37]:
data = np.load("../results/au6_or_l4/gas_FeH_abundance_evol.npy")
make_plot(data=data, xrange=(0.0, 14.0), yrange=(-2.5, 1.5),
          ylabel="[Fe/H]", label="Au6", filename="au6_FeH_abundance_evol")

In [40]:
data = np.load("../results/au6_or_l4/gas_OFe_abundance_evol.npy")
make_plot(data=data, xrange=(0.0, 14.0), yrange=(0.0, 1.0),
          ylabel="[O/Fe]", label="Au6", filename="au6_OFe_abundance_evol")